In [1]:
from utils import *
%load_ext autoreload
%autoreload 2

In [2]:
data = prepare_mnist_data()
data.keys()

dict_keys(['trainX', 'trainY', 'testX', 'testY'])

In [3]:
X = data["trainX"]
y = data["trainY"]

In [24]:
from MySolution import MyClustering

In [25]:
c = MyClustering(3)
c.train(X, verbose=True)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
loss:  27917709.0


KeyboardInterrupt: 

In [20]:
c.W

In [25]:
set(y)

{2.0, 5.0, 6.0}